In [ ]:
import pandas as pd
import xarray as xr
xr.set_options(keep_attrs=True,
               display_expand_data=False)

import plotly.express as px

from risk_config_port import portfolios
import risk_portfolios as p
from risk_chart import px_format


# Initialize parameters

In [ ]:
tickers = [ 'MWTIX', 'SPY', 'IWM', 'MDY', 'RSP', 'QQQ', 'XLK', 'XLI', 'XLF', 'XLC', 'XLE', 'XLY', 'XLB', 'XLV', 'XLU', 'XLP', 'VNQ', 'AIQ', 'ICLN', 'PFF', 'FEZ', 'EEM', 'FXI', 'ASHR',  'LQD', 'HYG', 'LQDH', 'HYGH', 'AGG',  'SHY', 'IEI', 'IEF', 'TLT', 'TIP', 'VTIP', 'AGNC', 'VMBS', 'CMBS', 'EMB', 'EMHY', 'GLD', 'SLV', 'USO', 'DBC', 'UUP', 'FXE', 'FXY' ]
returns = p.get_returns_basic(tickers)
rebalancing_dates = returns.resample('M').last().index

# Initialize a DataFrame to store portfolio weights in long format
portfolio_weights_long = pd.DataFrame(columns=["portfolio_name", "date", "ticker", "weight"])


# Run portfolio construction

In [ ]:
from importlib import reload
reload(p)
from risk_portfolios import PORTFOLIO_FUNCTIONS

portfolio_tuple = p.build_all_portfolios(portfolios, returns, rebalancing_dates)
returns, portfolio_weights_long = portfolio_tuple

# Observe output

In [ ]:
df = returns[portfolios.keys()].cumsum() 
px.line(df, template='plotly_white')

In [ ]:


def draw_portfolio_weights(portfolio_weights_long):
    ser = portfolio_weights_long.set_index(['date', 'portfolio_name', 'ticker'])
    da = ser.to_xarray()['weight']

    fig = (px.imshow(da.transpose(),
                     facet_col='portfolio_name',
                     facet_col_wrap=1,
                     height=1200,
                     zmin=-1,
                     zmax=+1,
                     color_continuous_scale='RdBu',
                     template='plotly_white')
           .for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1].strip()))
           .for_each_annotation(lambda a: a.update(text='') if a.x == 0 else None)
           .update_layout(coloraxis_showscale=False,
                          xaxis_title=None,
                          yaxis_title=None))
    return fig

fig = draw_portfolio_weights(portfolio_weights_long)
fig